# Day 14
## part one
polymerization, with a trick

In [1]:
package require Tcl 8.5

8.6.11

In [2]:
package require struct::list
package require math::linearalgebra

1.1.6

In [3]:
set tmp [struct::list filter [split "NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C" \n] {string length} ]

proc make_transition_matrix { pattern rulelist outmatrix outvector outmapping } {
    lappend rulelist "[string index $pattern end]% -> %"
    set pattern $pattern%

    upvar $outmapping map
    set map {}
    set tmp_mat {}
    foreach r $rulelist {
        lassign [split $r {-> }] rule _ _ _ prod 
        lassign [split $rule {}] pre post
        set row {}
        foreach atom [list $rule $pre$prod $prod$post] {
            set idx [lsearch -exact $map $atom]
            if { $idx < 0 } { lappend map $atom; set idx [expr {[llength $map]-1}] }
            lappend row $idx
        }
        lappend tmp_mat $row
    }


    upvar $outvector vec
    set vec [make_input_vector $pattern $map]


    set tmp_mat [lsort -integer -index 0 $tmp_mat]
    set cardinality [llength $vec]
    upvar $outmatrix mat
    set mat [math::linearalgebra::mkMatrix $cardinality $cardinality 0]
    for { set i 0 } { $i < $cardinality } { incr i } {
        set pre_production [lsearch -exact -all -inline -index 1 $tmp_mat $i]
        set post_production [lsearch -exact -all -inline -index 2 $tmp_mat $i]
        foreach precursor [list {*}$pre_production {*}$post_production] {
            lassign $precursor col
            math::linearalgebra::setelem mat $i $col [expr {[math::linearalgebra::getelem $mat $i $col] +1}]
        }
    }
}

proc make_input_vector { pattern mapping } {
    set card [llength $mapping]
    set vec [lrepeat $card 0]
    for { set i 0 } { $i < ([string length $pattern] -1)} {incr i} {
        set pair [string range $pattern $i [expr {$i +1}]]
        set idx [lsearch -exact $mapping $pair]
        lset vec $idx [expr { [lindex $vec $idx] + 1}]
    }
    return $vec
}

proc letters_population { state_vec mapping } {
    set pop [dict create]
    set letters [split [join $mapping ""] {}]
    foreach l $letters {
        dict set pop $l 0
    }
    foreach s $state_vec m $mapping {
        lassign [split $m {}] l
        dict incr pop $l [expr {int($s)}]
    }

    set res_pop {}
    dict for {l s} $pop {
        if { $l == {%} } { continue }
        lappend res_pop [list $l $s]
    }

    lsort -integer -index 1 $res_pop
}

make_transition_matrix [lindex $tmp 0] [lrange $tmp 1 end] test_matrix test_pattern test_mapping
puts "$test_pattern \n\n[math::linearalgebra::show $test_matrix {%1.0f}]\n$test_mapping"

0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 

0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 
1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 
0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 

CH CB BH HH HN NH HB NC HC BC CN NN NB BB BN CC B% %%


In [4]:
proc matpower { mat intpow } {
    if { $intpow <= 0 } { error "sorry only positive" }
    set res $mat
    for { set i 0 } { $i < $intpow } { incr i } {
        set res [::math::linearalgebra::matmul $mat $res]
    }
    return $res
}

In [7]:
set test_state [math::linearalgebra::matmul [matpower $test_matrix 9] $test_pattern]
set pop [letters_population $test_state $test_mapping]
set final_score [expr { [lindex $pop end 1] - [lindex $pop 0 1] }]
puts "$test_state\n$test_mapping\n$pop\n$final_score"

{C 298} {H 161} {B 1749} {N 865}
21.0 115.0 81.0 32.0 27.0 27.0 26.0 42.0 76.0 120.0 102.0 0.0 796.0 812.0 735.0 60.0 1.0 1.0
CH CB BH HH HN NH HB NC HC BC CN NN NB BB BN CC B% %%
{H 161} {C 298} {N 865} {B 1749}
1588


In [10]:
set fl [open 14input]
set input [struct::list filter [split [read $fl] \n] {string length} ]; close $fl

make_transition_matrix [lindex $input 0] [lrange $input 1 end] matrix pattern mapping
set state [math::linearalgebra::matmul [matpower $matrix 9] $pattern]
set pop [letters_population $state $mapping]
set final_score [expr { [lindex $pop end 1] - [lindex $pop 0 1] }]
puts "$state\n$pop\n$final_score"

{N 3108} {S 1749} {H 2184} {F 1522} {O 506} {P 2528} {C 2366} {V 1492} {K 2472} {B 1530}
106.0 187.0 572.0 0.0 61.0 0.0 42.0 472.0 30.0 704.0 627.0 262.0 288.0 249.0 187.0 285.0 202.0 398.0 218.0 52.0 21.0 106.0 115.0 673.0 362.0 301.0 655.0 153.0 0.0 0.0 243.0 1.0 191.0 441.0 240.0 316.0 442.0 0.0 371.0 279.0 212.0 126.0 0.0 91.0 799.0 135.0 66.0 43.0 144.0 545.0 0.0 319.0 244.0 0.0 1.0 193.0 281.0 266.0 152.0 143.0 124.0 367.0 597.0 449.0 297.0 568.0 236.0 0.0 291.0 150.0 246.0 191.0 224.0 33.0 231.0 515.0 0.0 31.0 116.0 104.0 0.0 265.0 278.0 113.0 101.0 45.0 45.0 0.0 0.0 0.0 38.0 0.0 0.0 126.0 0.0 0.0 0.0 0.0 63.0 0.0 1.0 1.0
{O 506} {V 1492} {F 1522} {B 1530} {S 1749} {H 2184} {C 2366} {K 2472} {P 2528} {N 3108}
2602


## part two
now do 40 iterations

In [12]:
set further_state [math::linearalgebra::matmul [matpower $matrix 29] $state]
set pop [letters_population $further_state $mapping]
set final_score [expr { [lindex $pop end 1] - [lindex $pop 0 1] }]
puts "$further_state\n$pop\n$final_score"

{N 3411547316650} {S 1937055921414} {H 2318406267841} {F 1541633298717} {O 468661394477} {P 2671034416074} {C 2481750112848} {V 1723833896052} {K 2711448364276} {B 1625349939396}
111670513583.0 166880869144.0 578738714182.0 0.0 53828677285.0 0.0 32474287045.0 479155339978.0 32472731454.0 854148914482.0 717900359984.0 273045427200.0 298734179439.0 263137620018.0 186686866432.0 272521341070.0 186693860981.0 446682268179.0 223341832607.0 50258346474.0 25130845619.0 93349301418.0 100504821477.0 701156286055.0 380373609258.0 384414096440.0 768830183551.0 192208496422.0 0.0 0.0 258135361096.0 1.0 204695272551.0 446872586009.0 253231339868.0 324424664588.0 495208555465.0 0.0 399505106657.0 316454605764.0 238362708423.0 158226879683.0 0.0 76071465329.0 871199462408.0 129367848882.0 86248055732.0 38035057158.0 136261707546.0 632906301131.0 0.0 320546852560.0 258134500917.0 0.0 1.0 190186507404.0 289368214190.0 306798803545.0 160272545328.0 144683368152.0 145070293314.0 387864017417.0 7009171831